# Single-shot communication task $C_{4,2,1}$
All examples regarding the task $C_{4,2,1}$ are collected to this notebook.\
First we import the Python libraries numpy and picos

In [1]:
import numpy as np
import picos as pc

Next we initialize some variables and lists which will be used to formulate SDPs

In [2]:
X = 6            # four settings for the preparation device
Y = 4            # six settings for the measurement device
K = 3            # binary outcomes
O = 2*Y*(K-1)+1  # total number of operators in our list
conP = []        # an empty list for constraints on the moment matrix level
conM = []        # an empty list for constraints on the substrate level
Prob = []        # a cell for probabilities
S = []           # an empty list for upper bounds
G = []           # cell for X moment matrices

Next step is to write down all constraints for the SDP

In [3]:
for x in range(X):
    G.append(pc.HermitianVariable("Moment_matrix_{}".format(x), (O, O))) # declare hermitian SDP variables
    conP.append(G[x] >> 0) # Semi-definiteness contraint
conP.append(G[0] + G[1] == G[2] + G[3]) # preparation equivalence
conP.append(G[0] + G[1] == G[4] + G[5]) # preparation equivalence

# function to return the position of the operators
def idx(y, k, u):
    return 2*(K-1)*y + 2*k + u + 2 - 1

for x in range(X):
    for y in range(Y):
        for k in range(0, K-1):
            conM.append(G[x][(0, idx(y,k,0))] == G[x][(idx(y,k,1), 0)])
            conM.append(G[x][(idx(y,k,0), 0)] == G[x][(0, idx(y,k,1))])
    for j in range(O):
        conM.append(G[x][(j, j)] == 1) # unitary constraints

We must then write the probabilities as elements of the moment matrices

In [4]:
#Obtain observed data
x_l = []
for x in range(X):
    y_l = []
    for y in range(Y):
        k_l = []
        for k in range(0, K-1):
            k_l.append(0.5 + 0.25 * (G[x][(0, idx(y,k,0))] + G[x][(0, idx(y,k,1))]))
        y_l.append(k_l)
    x_l.append(y_l)
Prob = x_l

We are now ready to solve the SDP. Result will be an optimal noncontextual bound for the success probability of the task $C_{4,2,1}$. \
The main part is forming the objective function $S$. 

In [5]:
S1 = (1 - (Prob[0][2][0] + Prob[0][2][1]) + 1 - (Prob[0][3][0] + Prob[0][3][1]) + 
      1 - (Prob[1][1][0] + Prob[1][1][1]) + Prob[1][3][1] +
      Prob[2][1][1] + Prob[2][2][1] + 1 - (Prob[3][0][0] + Prob[3][0][1]) + Prob[3][3][0] +
      Prob[4][0][1] + Prob[4][2][0] + Prob[5][0][0] + Prob[5][1][0]).real
P = pc.Problem()
P.set_objective("max", S1)
P.add_list_of_constraints(conM)
P.add_list_of_constraints(conP)
P.solve(solver = "cvxopt")
print(S1.value)


10.619026592454045


### Next we will use the internal see-saw method to inner estimate the quantum success probability

In [6]:
import itertools as it
import more_itertools as mit
from pprint import pprint
from qutip import *
import random

In [7]:
#definitions of random state generating functions

def random_rho(A=3, rank_lmt=3):
    a_ket = rand_ket(N=A, density=1, dims=None, seed=None)

    a_dm = a_ket * a_ket.dag()

    dms = [a_dm]

    total_dim = A
    for i in range(rank_lmt):
        a_ket = rand_ket(N=A, density=1, dims=None, seed=None)
        #print(a_ket)
        #print(np.linalg.norm(a_ket))
        #die
        a_dm = np.array(a_ket.data @ np.conj(a_ket.data).T)
        dms.append(a_dm)

    convex_weights = np.random.normal(size=len(dms))
    convex_weights = convex_weights / np.linalg.norm(convex_weights)
    convex_weights = np.array([x**2 for x in convex_weights])

    total_dm = sum([convex_weights[i] * dms[i] for i in range(len(dms))])

    return np.array(total_dm)
'''
def random_rho(A=3, rank_lmt=3):
    return np.array(rand_dm_ginibre(A, rank=rank_lmt))
'''
#Pauli matrices
#s = [sigmax(), sigmay(), sigmaz()]
s = {0: sigmax(), 1: sigmay(), 2: sigmaz()}

#General qubit state, input as list of Bloch vector components, i.e. r = [rx, ry, rz]
def rho(r):
    if np.linalg.norm(r) != 1:
        r = np.array(r)/np.linalg.norm(r)
    return np.array((qeye(2) + sum([r[i] * s[i] for i in range(3)])) / 2)

In the next cell the see-saw algorithm is defined

In [8]:
def see_saw(dim=2, num_of_states=4, rank_of_states=1, num_of_mes=6, num_of_outcomes=2, tol=10e-10):
    
    value1 = 1
    value2 = 2
    steps = 0

    tol = tol
    dim = dim

    #c_421
    r1 = random_rho(A=dim, rank_lmt=rank_of_states)

    x = [random_rho(A=dim, rank_lmt=rank_of_states) for i in range(num_of_states)]

    x = [pc.Constant(r) for r in x]
    I = pc.Constant(np.eye(dim))
    
    while value2 - value1 >= tol:

        x = [pc.Constant(r.value) for r in x]

        M = []
        for j in range(num_of_mes):
            Mj = []
            for k in range(num_of_outcomes - 1):
                Mj.append(pc.HermitianVariable("M_{}_{}".format(j, k), (dim,dim)))
            Mj.append(I - sum(mo for mo in Mj))
            M.append(Mj)
        
        
        
        P = pc.Problem()
        P.set_objective("max",  (x[0] | M[2][2]) + (x[0] | M[3][2]) + (x[1] | M[1][2]) + (x[1] | M[3][1]) + 
                                (x[2] | M[1][1]) + (x[2] | M[2][1]) + (x[3] | M[0][2]) + (x[3] | M[3][0]) +
                                (x[4] | M[0][1]) + (x[4] | M[2][0]) + (x[5] | M[0][0]) + (x[5] | M[1][0]))

        for m in M:
            P.add_list_of_constraints([mj >> 0 for mj in m])
            P.add_constraint(sum(m) == I)

        P.solve(solver = "cvxopt")

        value1 = pc.value((x[0] | M[2][2]) + (x[0] | M[3][2]) + (x[1] | M[1][2]) + (x[1] | M[3][1]) + 
                                (x[2] | M[1][1]) + (x[2] | M[2][1]) + (x[3] | M[0][2]) + (x[3] | M[3][0]) +
                                (x[4] | M[0][1]) + (x[4] | M[2][0]) + (x[5] | M[0][0]) + (x[5] | M[1][0])).real

        for j in range(len(M)):
            Mj = [pc.Constant(np.array(pc.value(m))) for m in M[j]]
            M[j] = Mj
            
        x = [pc.HermitianVariable("r{}".format(i), dim) for i in range(num_of_states)] 

        P = pc.Problem()
        P.set_objective("max",  (x[0] | M[2][2]) + (x[0] | M[3][2]) + (x[1] | M[1][2]) + (x[1] | M[3][1]) + 
                                (x[2] | M[1][1]) + (x[2] | M[2][1]) + (x[3] | M[0][2]) + (x[3] | M[3][0]) +
                                (x[4] | M[0][1]) + (x[4] | M[2][0]) + (x[5] | M[0][0]) + (x[5] | M[1][0]))


        P.add_list_of_constraints([r >> 0 for r in x])
        P.add_list_of_constraints([pc.trace(r) == 1 for r in x])
        #Contextual equivalence
        P.add_constraint(x[0]/2 + x[1]/2 == x[2]/2 + x[3]/2)
        P.add_constraint(x[2]/2 + x[3]/2 == x[4]/2 + x[5]/2)

        P.solve(solver = "cvxopt")

        value2 = pc.value((x[0] | M[2][2]) + (x[0] | M[3][2]) + (x[1] | M[1][2]) + (x[1] | M[3][1]) + 
                                (x[2] | M[1][1]) + (x[2] | M[2][1]) + (x[3] | M[0][2]) + (x[3] | M[3][0]) +
                                (x[4] | M[0][1]) + (x[4] | M[2][0]) + (x[5] | M[0][0]) + (x[5] | M[1][0])).real

        steps += 1
        
    return value2, value2 - value1, steps, x, M

In [9]:
ss = see_saw(dim=2, num_of_states=6, rank_of_states=10, num_of_mes=4, num_of_outcomes=3, tol=10e-10)
print(np.around(ss[:3], 10))
print(ss[:3])
print(ss[2])

for i in range(len(ss[3])):
    print("State {}:".format(i))
    print(np.around(ss[3][i].value, 3))
    
for i in range(len(ss[4])):
    print("POVM {}:".format(i))
    for j in range(len(ss[4][i])):
        print("Effect {}:".format(j))
        print(np.around(ss[4][i][j].value, 3))
        
for i in range(len(ss[3])):
    for j in range(len(ss[3])):
        print(i, j)
        print(np.trace(ss[3][i].value * ss[3][j].value))

[ 7.99999998e+00 -4.00000000e-10  3.20000000e+01]
(7.9999999772193675, -4.1159342600849413e-10, 32)
32
State 0:
[[ 0.432+0.j    -0.056+0.492j]
 [-0.056-0.492j  0.568+0.j   ]]
State 1:
[[0.568+0.j    0.056-0.492j]
 [0.056+0.492j 0.432+0.j   ]]
State 2:
[[0.587+0.j    0.489-0.059j]
 [0.489+0.059j 0.413+0.j   ]]
State 3:
[[ 0.413+0.j    -0.489+0.059j]
 [-0.489-0.059j  0.587+0.j   ]]
State 4:
[[0.568+0.j    0.056-0.492j]
 [0.056+0.492j 0.432+0.j   ]]
State 5:
[[ 0.432+0.j    -0.056+0.492j]
 [-0.056-0.492j  0.568+0.j   ]]
POVM 0:
Effect 0:
[[ 0.432+0.j    -0.056+0.492j]
 [-0.056-0.492j  0.568+0.j   ]]
Effect 1:
[[0.568+0.j    0.056-0.492j]
 [0.056+0.492j 0.432+0.j   ]]
Effect 2:
[[ 0.+0.j -0.+0.j]
 [-0.-0.j  0.+0.j]]
POVM 1:
Effect 0:
[[ 0.432+0.j    -0.056+0.492j]
 [-0.056-0.492j  0.568+0.j   ]]
Effect 1:
[[0.+0.j 0.-0.j]
 [0.+0.j 0.+0.j]]
Effect 2:
[[0.568+0.j    0.056-0.492j]
 [0.056+0.492j 0.432+0.j   ]]
POVM 2:
Effect 0:
[[0.568+0.j    0.056-0.492j]
 [0.056+0.492j 0.432+0.j   ]]
Effect

The value of between the see-saw method and the unitary hirarchy can be now compared. If result is the same, it means that an optimal quantum implementation has been found.

In [193]:
print(S1.value)
print(ss[0])
print(S1.value - ss[0])

10.619026592454045
7.999999984490245
2.6190266079638


Next we will calculate the noncontextual limit by constructin the matrix M and optimizing over the linear program \
There are 64 variables in the measurement-assignment polytope and there are 4 input states, so in total there are 256 variables $\nu_{P_j}(\kappa)$

In [ ]:
M = []
b = []
lhs = []
#Normalization
for k in range(4):
    r = []
    for i in range(256):
        if k*64 <= i < (k+1)*64:
            r.append(1)
        else:
            r.append(0)
    M .append(r)
    lhs.append(r)
    b.append(1)

#Contextual equivalence of preparations P1 + P2 == P3 + P4  

for i in range(64):
    r = []
    for j in range(256):
        if j == i + 0*64 or j == i + 1*64:
            r.append(1)
        elif j == i + 2*64 or j == i + 3*64:
            r.append(-1)
        else:
            r.append (0)
    M.append(r)
    lhs.append(r)
    b.append(0)


In [ ]:
#Here are the vertices of the measurement-assignment polytope
xi_imk = """(  1) 0 1 0 1 0 1 0 1 0 1 0 1 
(  2) 0 1 0 1 0 1 0 1 0 1 1 0 
(  3) 0 1 0 1 0 1 0 1 1 0 0 1 
(  4) 0 1 0 1 0 1 0 1 1 0 1 0 
(  5) 0 1 0 1 0 1 1 0 0 1 0 1 
(  6) 0 1 0 1 0 1 1 0 0 1 1 0 
(  7) 0 1 0 1 0 1 1 0 1 0 0 1 
(  8) 0 1 0 1 0 1 1 0 1 0 1 0 
(  9) 0 1 0 1 1 0 0 1 0 1 0 1 
( 10) 0 1 0 1 1 0 0 1 0 1 1 0 
( 11) 0 1 0 1 1 0 0 1 1 0 0 1 
( 12) 0 1 0 1 1 0 0 1 1 0 1 0 
( 13) 0 1 0 1 1 0 1 0 0 1 0 1 
( 14) 0 1 0 1 1 0 1 0 0 1 1 0 
( 15) 0 1 0 1 1 0 1 0 1 0 0 1 
( 16) 0 1 0 1 1 0 1 0 1 0 1 0 
( 17) 0 1 1 0 0 1 0 1 0 1 0 1 
( 18) 0 1 1 0 0 1 0 1 0 1 1 0 
( 19) 0 1 1 0 0 1 0 1 1 0 0 1 
( 20) 0 1 1 0 0 1 0 1 1 0 1 0 
( 21) 0 1 1 0 0 1 1 0 0 1 0 1 
( 22) 0 1 1 0 0 1 1 0 0 1 1 0 
( 23) 0 1 1 0 0 1 1 0 1 0 0 1 
( 24) 0 1 1 0 0 1 1 0 1 0 1 0 
( 25) 0 1 1 0 1 0 0 1 0 1 0 1 
( 26) 0 1 1 0 1 0 0 1 0 1 1 0 
( 27) 0 1 1 0 1 0 0 1 1 0 0 1 
( 28) 0 1 1 0 1 0 0 1 1 0 1 0 
( 29) 0 1 1 0 1 0 1 0 0 1 0 1 
( 30) 0 1 1 0 1 0 1 0 0 1 1 0 
( 31) 0 1 1 0 1 0 1 0 1 0 0 1 
( 32) 0 1 1 0 1 0 1 0 1 0 1 0 
( 33) 1 0 0 1 0 1 0 1 0 1 0 1 
( 34) 1 0 0 1 0 1 0 1 0 1 1 0 
( 35) 1 0 0 1 0 1 0 1 1 0 0 1 
( 36) 1 0 0 1 0 1 0 1 1 0 1 0 
( 37) 1 0 0 1 0 1 1 0 0 1 0 1 
( 38) 1 0 0 1 0 1 1 0 0 1 1 0 
( 39) 1 0 0 1 0 1 1 0 1 0 0 1 
( 40) 1 0 0 1 0 1 1 0 1 0 1 0 
( 41) 1 0 0 1 1 0 0 1 0 1 0 1 
( 42) 1 0 0 1 1 0 0 1 0 1 1 0 
( 43) 1 0 0 1 1 0 0 1 1 0 0 1 
( 44) 1 0 0 1 1 0 0 1 1 0 1 0 
( 45) 1 0 0 1 1 0 1 0 0 1 0 1 
( 46) 1 0 0 1 1 0 1 0 0 1 1 0 
( 47) 1 0 0 1 1 0 1 0 1 0 0 1 
( 48) 1 0 0 1 1 0 1 0 1 0 1 0 
( 49) 1 0 1 0 0 1 0 1 0 1 0 1 
( 50) 1 0 1 0 0 1 0 1 0 1 1 0 
( 51) 1 0 1 0 0 1 0 1 1 0 0 1 
( 52) 1 0 1 0 0 1 0 1 1 0 1 0 
( 53) 1 0 1 0 0 1 1 0 0 1 0 1 
( 54) 1 0 1 0 0 1 1 0 0 1 1 0 
( 55) 1 0 1 0 0 1 1 0 1 0 0 1 
( 56) 1 0 1 0 0 1 1 0 1 0 1 0 
( 57) 1 0 1 0 1 0 0 1 0 1 0 1 
( 58) 1 0 1 0 1 0 0 1 0 1 1 0 
( 59) 1 0 1 0 1 0 0 1 1 0 0 1 
( 60) 1 0 1 0 1 0 0 1 1 0 1 0 
( 61) 1 0 1 0 1 0 1 0 0 1 0 1 
( 62) 1 0 1 0 1 0 1 0 0 1 1 0 
( 63) 1 0 1 0 1 0 1 0 1 0 0 1 
( 64) 1 0 1 0 1 0 1 0 1 0 1 0 """
xi = xi_imk.split("\n")
#print(xi)
xi_l = []
for i in range(len(xi)):
    r = xi[i]
    r = r[r.find(')') + 1:]
    r = r.split(" ")[1:-1]
    r = [int(i) for i in r]
    xi_l.append(r)
    
#xi_l = np.array(xi_l)

In [ ]:
#Operational probabilities
M_n = {}
for i in range(6):
    r_xi = [x[i*2] for x in xi_l]
    for j in range(4):
        if j == 0:
            r = r_xi + [0 for k in range(3*64)]
        elif j == 1:
            r = [0 for k in range(64)] + r_xi + [0 for k in range(2*64)]
        elif j == 2:
            r = [0 for k in range(2*64)] + r_xi + [0 for k in range(1*64)]
        else:
            r = [0 for k in range(3*64)] + r_xi
        #M.append(r)
        M_n[(j, i)] = np.array(r)
        M.append(r)
        
#print(M[-20])
#print(M_n)

In [ ]:
#M matrix is now constructed. Now to construct the objective function
#Actually the objective function is directly a sum of the operational probabilities, and hence rows of the matrix M


'''value2 = pc.value((-x[0] | M[2][0]) - (x[0] | M[1][0]) - (x[0] | M[0][0]) -
                        (x[1] | M[4][0]) - (x[1] | M[3][0]) + (x[1] | M[0][0]) -
                        (x[2] | M[5][0]) + (x[2] | M[3][0]) + (x[2] | M[1][0]) +
                        (x[3] | M[5][0]) + (x[3] | M[4][0]) + (x[3] | M[2][0])).real'''

from scipy.optimize import linprog

obj = -(-M_n[(0, 2)] - M_n[(0, 1)] - M_n[(0, 0)] -
        M_n[(1, 4)] - M_n[(1, 3)] + M_n[(1, 0)] -
        M_n[(2, 5)] + M_n[(2, 3)] + M_n[(2, 1)] +
        M_n[(3, 5)] + M_n[(3, 4)] + M_n[(3, 2)])

lhs_eq = lhs
rhs_eq = b

bnd = [(0,1) for i in range(256)]

opt = linprog(c = obj, A_eq = lhs_eq, b_eq=rhs_eq, bounds=bnd)
print(opt)
print(-opt['fun'])

In [ ]:
for i in range(6):
    r_xi = [x[i*2] for x in xi_l]
    for j in range(4):
        if j == 0:
            r = r_xi + [0 for k in range(3*64)]
        elif j == 1:
            r = [0 for k in range(64)] + r_xi + [0 for k in range(2*64)]
        elif j == 2:
            r = [0 for k in range(2*64)] + r_xi + [0 for k in range(1*64)]
        else:
            r = [0 for k in range(3*64)] + r_xi
        #M.append(r)
        print(np.array(r)@opt['x'])
        
print(obj@opt['x'])

So the noncontextual limit seems to be exactly 4. We can still find out the noncontextual inequality that is most violated.

In [ ]:
#Observed probabilities
'''S1 = (-Prob[0][0][0] - Prob[0][1][0] - Prob[0][2][0]
      +Prob[1][0][0] - Prob[1][3][0] - Prob[1][4][0]
      +Prob[2][1][0] + Prob[2][3][0] - Prob[2][5][0]
      +Prob[3][2][0] + Prob[3][4][0] + Prob[3][5][0]).real
      '''
print(Prob[0][0][0].real)
b_nonc = b.copy()
b_star = b.copy()
b_text = []
for j in range(6):
    for i in range(4):
        b_star.append((Prob[i][j][0].real).value)
        b_nonc.append(M_n[(i, j)]@opt['x'])
        b_text.append("P(0 | M_{}, P_{})".format(j, i))
        

        

In [ ]:
#Farkas' dual
Mt = np.transpose(M)
#Mt = np.copy(M)
Mt = [list(row) for row in Mt]
#print(Mt)
#print(b_star)
#print(b_nonc)

obj = b_star

print(len(Mt))
zeros = [0 for i in range(len(Mt))]
ones = [1 for i in range(len(Mt))]
print(len(zeros))
print(len(ones))

lhs_ineq = Mt.copy()
for i in range(len(lhs_ineq)):
    lhs_ineq.append([-j for j in lhs_ineq[i]])


rhs_ineq = ones
for i in range(len(zeros)):
    rhs_ineq.append(0)

#bnd = [(-float("inf"),float("inf")) for i in range(len(b_star))]
bnd = [(-1,1) for i in range(len(b_star))]

opt2 = linprog(c = obj, A_ub=lhs_ineq, b_ub=rhs_ineq, bounds=bnd, method='interior-point')
print(opt2)
print(len(opt2['x']))
print(len(b_star[-24:]))
print(opt2['x'][-24:])

for i in range(0, len(b_star)-24):
    if b_star[i] != 0:
        print(i)
        print(b_star[i])
        print(opt2['x'][i])
        print(b_star[i]*opt2['x'][i])
        
for i in range(len(b_star)-24, len(b_star)):
    if b_star[i] != 0:
        print(i)
        #print(b_star[i])
        print(opt2['x'][i])
        #print(b_star[i]*opt2['x'][i])

In [ ]:
s = ""
su = 0
sum_co = 0
for i in range(len(b_star)-24, len(b_star)):
    sign = " +" if np.sign(-opt2['x'][i]) == 1 else " "
    s += sign + str("%.6f" % -opt2['x'][i]) + "*" + b_text[i - len(b_star) + 24]
    su += -opt2['x'][i]*b_star[i]
    sum_co += -opt2['x'][i]
print(s)
print(su)
print(np.sqrt(2) - 1)
print(sum_co)